# 2 Descarga una iamgen LS2 de sogamoso para desplegarla en el QGIS 

In [ ]:
// Definir la región de interés (Sogamoso, Boyacá, Colombia)
var sogamoso = ee.Geometry.Point([-72.9286, 5.7143]);

// Filtrar la colección de imágenes Landsat 8 SR (sin filtrar por nubosidad aún)
var landsatCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                          .filterBounds(sogamoso) 
                          .filterDate('2022-01-01', '2024-02-01'); // Se amplía el rango de fechas

// Contar imágenes disponibles
var count = landsatCollection.size();
print("Número de imágenes disponibles:", count);

// Mostrar lista de imágenes con fechas y cobertura de nubes
var imageList = landsatCollection.map(function(image) {
  return ee.Feature(null, {
    'ID': image.id(),
    'Fecha': image.date().format('YYYY-MM-dd'),
    'Nubosidad': image.get('CLOUD_COVER')
  });
});

print("Lista de imágenes encontradas:", imageList);

// Si hay imágenes disponibles, procesamos la mejor
if (count.gt(0)) {
  var bestImage = landsatCollection.sort('CLOUD_COVER').first(); // Tomamos la de menor nubosidad
  print("Imagen seleccionada:", bestImage);

  // Seleccionar bandas de reflectancia superficial
  var bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']; 
  var scaledImage = bestImage.select(bands).multiply(0.0000275).add(-0.2); 

  // Visualizar en el mapa
  var visParams = {bands: ['SR_B4', 'SR_B3', 'SR_B2'], min: 0, max: 0.3, gamma: 1.4};
  Map.centerObject(sogamoso, 10);
  Map.addLayer(scaledImage, visParams, "Landsat 8 SR - Sogamoso");

  // Exportar la imagen a Google Drive
  Export.image.toDrive({
    image: scaledImage,
    description: "Landsat8_SR_Sogamoso",
    folder: "GEE_Exports",
    fileNamePrefix: "Landsat8_SR_Sogamoso",
    region: sogamoso.buffer(5000).bounds(),
    scale: 30,
    maxPixels: 1e13,
    fileFormat: "GeoTIFF"
  });

} else {
  print("❌ No se encontraron imágenes en el período seleccionado.");
}
